In [2]:
## Notebook to evaluate and store the results from ggcnn for a sample imageset
## Last Trial Run: 7th April 2021

In [2]:
import os 
import glob
import sys 
import torch
import numpy as np
import torch.utils.data 
import imageio
import pandas as pd
import time

In [3]:
ggcnn_dir = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/ggcnn_repo/"
fadi_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Experts/Generative/pretrained_weights"
sys.path.append(ggcnn_dir)
sys.path.append(fadi_dir)
from models.common import post_process_output
from utils.dataset_processing import evaluation, grasp 
from utils.data import get_dataset
from utils.dataset_processing.image import DepthImage

In [4]:
## Considering one folder!
dataset_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/"
dataset = get_dataset("cornell")
sys.path.insert(0, dataset_path)
test_data = dataset(dataset_path, include_depth = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/*/pcd*cpos.txt


In [6]:
# ## Considering one folder!
# folder1_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/"
# dataset = get_dataset("cornell")
# sys.path.insert(0, folder1_path)
# test_data = dataset(folder1_path, include_depth = True)


Search Path:  /media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/*/pcd*cpos.txt


In [5]:
len(test_data.depth_files)

885

In [6]:
## Model Loading:
%time
ggcnn2_model_dir = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/ggcnn_repo/weights/ggcnn2_weights_cornell/epoch_50_cornell"
ggcnn2_net = torch.load(ggcnn2_model_dir)
net = ggcnn2_net

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/torch/serialization.py

In [7]:
test_comp = test_data
test_data = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
# ##  Trial for each test image
# start_cell = time.time()
# results = {'correct': 0, 'failed': 0}
# iou_quality = {}
# with torch.no_grad():

#     ## For each test image
#     ## Just testing for one single image
#     x, y, didx, rot, zoom =  next(iter(test_data))
#     idx = 0
#     xc = x.to(device)
#     yc = [yi.to(device) for yi in y]
#     lossd = net.compute_loss(xc, yc)    ## Ensure that your outputs are from a dataloader!

#     q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
#     s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

#     if s:
#         results['correct'] += 1
#     else:
#         results['failed'] += 1

#     iou_dict = {}
#     ## Now to compare and create a dictionary that stores the quality given image name
#     x_comp, _, _, _, _ = test_comp[idx]
#     #print(torch.allclose(torch.tensor(x_comp), x))
#     if torch.allclose(torch.tensor(x_comp), x):
#         img_id = test_comp.depth_files[0].split("/")[-1]

#         ## We cannot use an image!
#         #quality_dict[str(img_id)] = q_img
#         image_id = str(img_id).split('.')[0][:-1]
#         iou_quality[image_id] = [int(s), np.max(q_img).astype('float32')]
#     else:
#         print("Dataloader and Comparison X didn't match")

# print("Cell Time: ", time.time() - start_cell)

Cell Time:  0.41224050521850586


In [9]:
results = {'correct': 0, 'failed': 0}
start_cell = time.time()
iou_quality = {}
with torch.no_grad():


    for idx, (x, y, didx, rot, zoom) in enumerate(test_data):
        xc = x.to(device)
        yc = [yi.to(device) for yi in y]
        lossd = net.compute_loss(xc, yc)

        q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])
        s = evaluation.calculate_iou_match(q_img, ang_img, test_data.dataset.get_gtbb(didx, rot, zoom), no_grasps = 1, grasp_width = width_img)

        if s:
            results['correct'] += 1
        else:
            results['failed'] += 1

        ## Need to store: q_img and img_id
        ## Now to compare and create a dictionary that stores the quality given image name
        x_comp, _, _, _, _ = test_comp[idx]
        
        ## Now, trying to 
        if torch.allclose(torch.tensor(x_comp), x):
            img_id = test_comp.depth_files[idx].split("/")[-1]
            img_id = str(img_id).split('.')[0][:-1]
            ## We cannot use an image!
            #quality_dict[str(img_id)] = q_img
            iou_quality[str(img_id)] = [int(s), np.max(q_img).astype('float32')]
        else:
            print("Dataloader and Comparison X didn't match")
print("Cell Time: ", time.time() - start_cell) ## ~90s

Cell Time:  89.68731474876404


In [13]:
# print(iou_quality)

{'pcd0100': [1, 0.78971946], 'pcd0101': [1, 0.8981491], 'pcd0102': [1, 0.7706125], 'pcd0103': [1, 0.9016831], 'pcd0104': [1, 0.7905614], 'pcd0105': [0, 0.85269386], 'pcd0106': [0, 0.12784792], 'pcd0107': [0, 0.07468596], 'pcd0108': [0, 0.0750214], 'pcd0109': [0, 0.0789577], 'pcd0110': [0, 0.952965], 'pcd0111': [1, 0.90971625], 'pcd0112': [0, 0.9798054], 'pcd0113': [0, 0.84903944], 'pcd0114': [0, 0.8830497], 'pcd0115': [0, 0.9404322], 'pcd0116': [1, 0.9564643], 'pcd0117': [1, 1.2627996], 'pcd0118': [0, 0.07337329], 'pcd0119': [1, 0.52589566], 'pcd0120': [1, 0.62857115], 'pcd0121': [0, 0.6219607], 'pcd0122': [1, 0.9120887], 'pcd0123': [0, 0.87171906], 'pcd0124': [1, 0.8176524], 'pcd0125': [0, 0.8733429], 'pcd0126': [0, 0.6590582], 'pcd0127': [0, 0.88319856], 'pcd0128': [0, 0.8827105], 'pcd0129': [0, 0.07505435], 'pcd0130': [0, 0.082016736], 'pcd0131': [0, 0.0959656], 'pcd0132': [0, 0.2175512], 'pcd0133': [0, 0.07445402], 'pcd0134': [0, 0.07422224], 'pcd0135': [0, 0.07516372], 'pcd0136': 

In [14]:
## To get the image id
img_id = test_comp.rgb_files[1].split("/")[-1]
depth_id = str(img_id).split('.')[0][:-1]
print(depth_id)

rgb_id = str(img_id).split('.')[0][:-1]
print(rgb_id)

pcd0101
pcd0101


In [10]:
## Saving the Data Collected
import pickle
model_name = "ggcnn2_depth"
save_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/"
first_entry, last_entry = list(iou_quality.keys())[0], list(iou_quality.keys())[-1]
fname = "_".join((model_name, first_entry, last_entry+".txt"))
fdir = os.path.join(save_dir, fname)
with open(fdir, 'wb') as file_pickle:
    pickle.dump(iou_quality, file_pickle)

In [11]:
fdir

'/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/ggcnn2_depth_pcd0100_pcd1034.txt'